#Import OpenAI SDK

In [1]:
# Install the openai-agents library
!pip install -Uq openai-agents

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.1/40.1 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.6/130.6 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.3/129.3 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.9/150.9 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 2.5 MB/s eta 0:00:00


#Import Asyncio to run asynchronus functions

In [2]:
# Import the nest_asyncio library
import nest_asyncio
# Apply nest_asyncio to allow asyncio to run in a nested environment (like Colab)
nest_asyncio.apply()

#Set Gemini API key as Default API reference instaed of Open API key and Setting Up Configurations

In [4]:
import os

from agents import Agent, Runner, AsyncOpenAI, OpenAIChatCompletionsModel
from agents.run import RunConfig
from google.colab import userdata

In [10]:
from agents import (
    Agent,
    Runner,
    function_tool,
    set_default_openai_api,
    set_default_openai_client,
    set_tracing_disabled,
)

BASE_URL = "https://generativelanguage.googleapis.com/v1beta/openai/"
API_KEY = userdata.get("GEMINI_API_KEY")
MODEL_NAME = "gemini-2.0-flash"


if not BASE_URL or not API_KEY or not MODEL_NAME:
    raise ValueError(
        "Please set EXAMPLE_BASE_URL, EXAMPLE_API_KEY, EXAMPLE_MODEL_NAME via env var or code."
    )



client = AsyncOpenAI(
    base_url=BASE_URL,
    api_key=API_KEY,
)

set_default_openai_client(client=client, use_for_tracing=True)
set_tracing_disabled(False)
set_default_openai_api("chat_completions")

#Tracing
##1. Local Tracing

In [11]:
import os
from openai import AsyncOpenAI
from agents import Agent, Runner, trace, set_default_openai_api, set_default_openai_client, set_trace_processors
from agents.tracing.processor_interface import TracingProcessor
from pprint import pprint
# Custom trace processor to collect trace data locally
class LocalTraceProcessor(TracingProcessor):
    def __init__(self):
        self.traces = []
        self.spans = []

    def on_trace_start(self, trace):
        self.traces.append(trace)
        print(f"Trace started: {trace.trace_id}")

    def on_trace_end(self, trace):
        print(f"Trace ended: {trace.export()}")

    def on_span_start(self, span):
        self.spans.append(span)
        print(f"Span started: {span.span_id}")
        print(f"Span details: ")
        pprint(span.export())

    def on_span_end(self, span):
        print(f"Span ended: {span.span_id}")
        print(f"Span details:")
        pprint(span.export())

    def force_flush(self):
        print("Forcing flush of trace data")

    def shutdown(self):
        print("=======Shutting down trace processor========")
        # Print all collected trace and span data
        print("Collected Traces:")
        for trace in self.traces:
            print(trace.export())
        print("Collected Spans:")
        for span in self.spans:
            print(span.export())

BASE_URL = os.getenv("BASE_URL") or "https://generativelanguage.googleapis.com/v1beta/openai/"
API_KEY = os.getenv("GEMINI_API_KEY") or userdata.get("GEMINI_API_KEY")
MODEL_NAME = os.getenv("MODEL_NAME") or "gemini-2.0-flash"


if not BASE_URL or not API_KEY or not MODEL_NAME:
    raise ValueError("Please set BASE_URL, GEMINI_API_KEY, MODEL_NAME via env var or code.")

# Create OpenAI client
client = AsyncOpenAI(
    base_url=BASE_URL,
    api_key=API_KEY,
)

# Configure the client
set_default_openai_client(client=client, use_for_tracing=True)
set_default_openai_api("chat_completions")

# Set up the custom trace processor
local_processor = LocalTraceProcessor()
set_trace_processors([local_processor])

# Example function to run an agent and collect traces
async def main():
    agent = Agent(name="Example Agent", instructions="Perform example tasks.", model=MODEL_NAME)

    with trace("Example workflow"):
        first_result = await Runner.run(agent, "Start the task")
        second_result = await Runner.run(agent, f"Rate this result: {first_result.final_output}")
        print(f"Result: {first_result.final_output}")
        print(f"Rating: {second_result.final_output}")

# Run the main function
import asyncio
asyncio.run(main())

Trace started: trace_2cf33c977f6841eb8500674e71f823fe
Span started: span_0226d93857f34b85a63efc10
Span details: 
{'ended_at': None,
 'error': None,
 'id': 'span_0226d93857f34b85a63efc10',
 'object': 'trace.span',
 'parent_id': None,
 'span_data': {'handoffs': [],
               'name': 'Example Agent',
               'output_type': 'str',
               'tools': None,
               'type': 'agent'},
 'started_at': '2025-07-05T10:48:09.935387+00:00',
 'trace_id': 'trace_2cf33c977f6841eb8500674e71f823fe'}
Span started: span_3de53f676ed743f08a3ad515
Span details: 
{'ended_at': None,
 'error': None,
 'id': 'span_3de53f676ed743f08a3ad515',
 'object': 'trace.span',
 'parent_id': 'span_0226d93857f34b85a63efc10',
 'span_data': {'input': None,
               'model': 'gemini-2.0-flash',
               'model_config': {'base_url': 'https://generativelanguage.googleapis.com/v1beta/openai/',
                                'extra_args': None,
                                'extra_body': None,
  